In [5]:
# Импортируем внешние библиотеки
import asyncio
import aiohttp
from sqlalchemy import BigInteger, String, Boolean, DateTime
# Импортируем библиотеки для работы с системой и путями
import os
from  pathlib import Path
import sys
# Установка базовой директории и пути к файлу с учетными данными. Используем конструкцию try-except для обработки возможных ошибок при определении пути для notebook.
try:
    BASE_DIR = Path(__file__).resolve().parents[2]
except NameError:
    BASE_DIR = Path.cwd().resolve().parents[1] 

SRC_DIR = os.path.join(BASE_DIR, 'src')

# Добавляем src в пути поиска модулей
if SRC_DIR not in sys.path:
    sys.path.append(SRC_DIR)

# Импорт самописных утилит
from core.utils_general import load_api_tokens
from core.utils_sql import get_db_engine, sync_data_to_postgres

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Определение пути к файлу с учетными данными для notebook. Если переменные окружения не установлены или указывают на несуществующий файл, будет выброшено исключение с информативным сообщением.
try:
    TOKENS_PATH = BASE_DIR / os.getenv("CREDS_DIR") / os.getenv('TOKENS_FILE')        
except TypeError:
    raise ValueError("Проверьте переменные окружения CREDS_DIR и CREDS_FILE. Они должны быть установлены и указывать на существующий файл с учетными данными.")

In [ ]:
async def get_advert_info_wb(account, token, session):
    """Асинхронная функция для получения информации о рекламных кампаниях c ВБ."""
    url = "https://advert-api.wildberries.ru/api/advert/v2/adverts"
    headers = {"Authorization": token}
    params = {"statuses": 9}
    
    try:
        async with session.get(url, headers=headers, params=params, timeout=10) as response:
            # 1. Сразу пытаемся распарсить JSON, если это возможно
            content_type = response.headers.get('Content-Type', '')
            data = await response.json() if 'application/json' in content_type else None
            
            if response.status == 200:
                data['account'] = account  # Добавляем информацию об аккаунте в данные
                print(f"✅ [{account}] Данные получены")
                return data
            
            # 2. Безопасно достаем описание ошибки
            error_detail = data.get('detail') if data else await response.text()
            
            # 3. Обработка ошибок без дублирования кода
            if response.status == 401:
                print(f"🔑 [{account}] Ошибка 401: Неверный токен. ({error_detail})")
            elif response.status == 429:
                print(f"⏳ [{account}] Ошибка 429: Лимит запросов! ({error_detail})")
            elif response.status == 400:
                print(f"❓ [{account}] Ошибка 400: Плохой запрос. ({error_detail})")
            else:
                print(f"❌ [{account}] Ошибка {response.status}: {error_detail}")
            
            return None
            
    except Exception as e:
        print(f"💥 [{account}] Непредвиденная ошибка: {e}")
        return None

In [ ]:
async def fetch_advert_info(tokens: dict):
    """Асинхронная функция для получения информации о рекламных кампаниях для всех аккаунтов и токенов."""
    # Асинхронно обрабатываем все аккаунты и токены
    async with aiohttp.ClientSession() as session:
            # Создаем задачи для каждого аккаунта и токена
            tasks = [get_advert_info_wb(account, token, session) for account, token in tokens.items()]
            # Ожидаем завершения всех задач и собираем результаты
            results = await asyncio.gather(*tasks)
            return results

In [ ]:
tokens = load_api_tokens()
advert_info_data = await fetch_advert_info()

✅ [СТАРТ9237] Данные получены
✅ [Оганесян] Данные получены
✅ [ВЕКТОР7794] Данные получены
✅ [ВЕКТОР4216] Данные получены
✅ [Хачатрян] Данные получены
✅ [Даниелян] Данные получены
✅ [Вектор2] Данные получены
✅ [Старт2] Данные получены
✅ [ВЕКТОР3522] Данные получены
✅ [Вектор] Данные получены
✅ [Пилосян] Данные получены
✅ [СТАРТ4040] Данные получены
✅ [Тоноян] Данные получены
✅ [Лопатина] Данные получены
✅ [СТАРТ8748] Данные получены
✅ [Старт] Данные получены


In [ ]:
def extract_campaign_info(advert_info_data):
    """
    Преобразует вложенную структуру данных API Wildberries в плоский список словарей.
    Это упрощает дальнейший анализ данных или их экспорт в Excel/Pandas.
    """
    # Инициализируем список для хранения финальной информации о рекламных кампаниях 
    campaign_info = []

    for account_data in advert_info_data:
        # ПРОВЕРКА: Если данные по аккаунту пустые или API вернул ошибку, пропускаем этот аккаунт
        if not account_data or 'adverts' not in account_data:
            continue
            
        # Добавляем информацию об аккаунте (берем из ключа, который мы добавили вручную в get_advert_info_wb)
        account_name = account_data.get('account', 'Unknown')
        
        # Цикл по каждой рекламной кампании (advert) внутри данных текущего аккаунта
        for advert in account_data.get('adverts', []):
            
            # БЕЗОПАСНОЕ ИЗВЛЕЧЕНИЕ ВЛОЖЕННЫХ ОБЪЕКТОВ:
            # Чтобы не писать длинные проверки на каждой строке, выносим вложенные словари в переменные.
            # Используем .get() и пустые словари {} / списки [{}], чтобы код не "падал", если WB не прислал ключи.
            
            # Извлекаем настройки товара (берем первый элемент списка nm_settings, если он существует)
            nm_settings = advert.get('nm_settings', [{}])[0] if advert.get('nm_settings') else {}
            
            # Извлекаем ставки в копейках (вложены в nm_settings)
            bids = nm_settings.get('bids_kopecks', {})
            
            # Извлекаем общие настройки (название, способ оплаты) и зоны показов (placements)
            settings = advert.get('settings', {})
            placements = settings.get('placements', {})
            
            # Формируем итоговый "плоский" словарь для текущей кампании
            campaign_info.append({
                'account': account_name,
                'campaign_id': advert.get('id'),
                'campaign_name': settings.get('name'),
                
                # Тип ставки (например, unified — единая ставка, manual — ручная ставка)
                'bid_type': advert.get('bid_type'),
                
                # ID товара (артикул WB)
                'nm_id': nm_settings.get('nm_id'),
                
                # Ставка в Поиске (извлекаем из bids_kopecks -> search)
                'search_bid': bids.get('search'),
                
                # Ставка в рекомендациях (извлекаем из bids_kopecks -> recommendations)
                'recommendations_bid': bids.get('recommendations'),
                
                # Тип оплаты (например, CPC — оплата за клик, CPM — оплата за 1000 показов)
                'payment_type': settings.get('payment_type'),
                
                # Статус показов в рекомендательных полках (True/False)
                'recommendations': placements.get('recommendations'),
                
                # Статус показов в поиске (True/False)
                'search': placements.get('search'),
                
                # Дата создания кампании (безопасно достаем из вложенного объекта timestamps)
                'created_at_campaign': advert.get('timestamps', {}).get('created')
            })
            
    return campaign_info

# Преобразуем данные из API в плоскую структуру
data = extract_campaign_info(advert_info_data)

# Установка соединения с базой данных
engine = get_db_engine()

In [ ]:


# Описываем типы данных для каждой колонки, чтобы при синхронизации с PostgreSQL данные корректно сохранялись в нужных форматах.
schema_definition = {
    'campaign_id': BigInteger,
    'campaign_name': String,
    'bid_type': String,
    'nm_id': BigInteger,
    'search_bid': BigInteger,
    'recommendations_bid': BigInteger,
    'payment_type': String,
    'recommendations': Boolean,
    'search': Boolean,
    'created_at_campaign': DateTime,
    'account': String
}

table_name = 'advert_campaigns_info'
unique_keys = ['campaign_id']  # Уникальность по ID кампании

sync_data_to_postgres(engine, table_name, data, schema_definition, unique_keys)

✅ Таблица 'advert_campaigns_info': успешно синхронизирована по ключам ['campaign_id']
✅ Таблица 'advert_campaigns_info': обработано 1376 строк.


In [6]:
import os

def print_project_tree(startpath, exclude_dirs=None):
    if exclude_dirs is None:
        exclude_dirs = {'.git', '__pycache__', '.ipynb_checkpoints', 'venv', '.env'}
    
    print(f"📁 Проект: {os.path.basename(os.path.abspath(startpath))}")
    for root, dirs, files in os.walk(startpath):
        # Фильтруем папки, которые не хотим видеть
        dirs[:] = [d for d in dirs if d not in exclude_dirs]
        
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print(f"{indent}├── {os.path.basename(root)}/")
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print(f"{subindent}└── {f}")

# Укажите путь к вашему BASE_DIR, если вы не в корне
# Если запустить прямо сейчас в корне, сработает '.'
print_project_tree(str(BASE_DIR)) # Или подставьте свой BASE_DIR

📁 Проект: start_vector
├── start_vector/
    └── .env
    └── requirements.txt
    ├── creds/
        └── creds.json
        └── tokens.json
    ├── logs/
    ├── src/
        ├── core/
            └── utils_general.py
            └── utils_sql.py
            └── __init__.py
        ├── modules/
            └── __init__.py
            ├── COMETA/
            ├── WB/
                ├── advert/
                    └── api.py
                    └── processing.py
                    └── schemas.py
        ├── notebooks/
            └── advert_info.ipynb
            └── __init__.py


In [19]:
advert_campaigns_info_dict = {
    "advert_campaigns_info": {
        'campaign_id': BigInteger,
        'campaign_name': String,
        'bid_type': String,
        'nm_id': BigInteger,
        'search_bid': BigInteger,
        'recommendations_bid': BigInteger,
        'payment_type': String,
        'recommendations': Boolean,
        'search': Boolean,
        'created_at_campaign': DateTime,
        'account': String
    }
}

table_name = list(advert_campaigns_info_dict.keys())[0]
unique_keys = ['campaign_id']  # Уникальность по ID кампании
schema_definition = advert_campaigns_info_dict.get('advert_campaigns_info')

unique_keys

['campaign_id']

In [20]:
table_name

'advert_campaigns_info'